In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import fitz
from requests.exceptions import HTTPError
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
import pandas as pd

df = pd.read_csv('scraped_police_reports14.csv')

In [ ]:
with open('serper_secrets.txt', 'r') as file:
    data=json.load(file)
    

In [ ]:
key=data["key"]
def web_search(officer, department):
    url = "https://google.serper.dev/search"

    name = namify(officer)
    department=departmentify(department)
    

    payload = json.dumps({
    "q": f'"{name}" "{department}"',
    "num":10
    })
    headers = {
    'X-API-KEY': '186ebef8bff1f77421b10617b1019e6ab2386b2a',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    data=response.json()
    q=data["organic"]
    return q

In [ ]:
j=df[df["url"]=="No articles found"]

In [ ]:
import re
def namify(title):
    match = re.search(r"^(.*?),", title)

    if match:
        result = match.group(1)
    return result

In [ ]:
def departmentify(department):
    department= department.replace("Office","")
    department = department.replace("Department", "")
    department = department.replace(":", "")
    return department

In [ ]:
new_articles=[]
#namify(j.iloc[5]["name"])
w=j.head()
for index, row in j.head(2000).iterrows():
    q=web_search(row["name"], row["department"])
    if len(q)==0:
        new_articles.append((row["name"],row["department"],"No articles found"))
    else:
        for p in q:
            new_articles.append((row["name"],row["department"],p.get("title","NA"), p.get("link", "NA"), p.get("snippet", "NA"), p.get("date", "NA")))

In [ ]:
dataf=pd.DataFrame(new_articles, columns=["name", "department", "title", "url", "summary", "date posted"])

In [ ]:
dataf.to_csv("serper_scrape.csv", index=False)